In [ ]:
# Import Libraries
!pip install psycopg2

import requests
import pandas as pd
import datetime
import psycopg2 as ps

In [ ]:
# Calling the API
page_Token = 'BQCJk9gExrMOyB8xWkJa1ZWlIeto-yFwxPZlx1hfRD0sX8gW1ASW68tLt-NAWdU_qzZMN1laSN9bTDBqkSrJNSlbGMy6X_1tYs9qrk3Lt3QA_BrdH47Gv7L6l0RvACwU6GmIhl16YCN9_FWEpKasWH-vBIT8STKcS1SEgjghdFLPFAIGhzZLP00qtSpjEAmTLHoVsg'
headers = {
    "Accept": "application/json",
    "Content_type": "application/json",
    "Authorization": "Bearer {token}".format(token=page_Token)
}

#Spotify uses unix timestamp
today = datetime.datetime.now()
yesterday = today - datetime.timedelta(hours=1)
yesterday_unix_timestamp = int(yesterday.timestamp())*1000

url_time= 'https://api.spotify.com/v1/me/player/recently-played?after={time}'.format(time=yesterday_unix_timestamp)
r_time = requests.get(url_time, headers=headers).json()
r_time['items'][0]['played_at']

In [ ]:
uri= []
song_names= []
artist_names= []
popularity= []

for song in r_time['items']:
    uri.append(song['track']['id'])
    song_names.append(song['track']['name'])
    artist_names.append(song['track']['artists'][0]['name'])
    popularity.append(song['track']['popularity'])

song_dict= {
    'id': uri,
    'song_names': song_names,
    'artist_names': artist_names,
    'uri': uri,
    'popularity': popularity,
}
    
# We drop duplicates based on uri column
song_df = pd.DataFrame(song_dict, columns=['uri', 'song_names', 'artist_names', 'popularity'])
song_df = song_df.drop_duplicates()
song_df

In [ ]:
BASE_URL = 'https://api.spotify.com/v1/'
feature_dict = {}
track_uris = song_df['uri'].to_list()

for t_uri in track_uris:
    
    feature_dict[t_uri] = {'popularity': 0,
                         'acousticness': 0,
                         'danceability': 0,
                         'duration_ms': 0,
                         'energy': 0,
                         'liveness': 0,
                         'loudness': 0,
                         'mode': 0,
                         'speechiness': 0,
                         'instrumentalness': 0,
                         'tempo': 0,
                         'valence': 0}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    feature_dict[t_uri]['popularity'] = r['popularity']
    
    s = requests.get(BASE_URL + 'audio-features/' + t_uri, headers=headers)
    s = s.json()
    feature_dict[t_uri]['acousticness'] = s['acousticness']
    feature_dict[t_uri]['danceability'] = s['danceability']
    feature_dict[t_uri]['duration_ms'] = s['duration_ms']
    feature_dict[t_uri]['energy'] = s['energy']
    feature_dict[t_uri]['liveness'] = s['liveness']
    feature_dict[t_uri]['loudness'] = s['loudness']
    feature_dict[t_uri]['mode'] = s['mode']
    feature_dict[t_uri]['speechiness'] = s['speechiness']
    feature_dict[t_uri]['instrumentalness'] = s['instrumentalness']
    feature_dict[t_uri]['tempo'] = s['tempo']
    feature_dict[t_uri]['valence'] = s['valence']

In [ ]:
df_features = pd.DataFrame.from_dict(feature_dict, orient='index')
df_features.insert(0, 'uri', df_features.index)
df_features.reset_index(inplace=True, drop=True)

df_features.head(20)

In [ ]:

#Vamos a unir las tablas
spotify_df = pd.merge(song_df, df_features)
spotify_df = spotify_df.drop('uri', axis=1)
spotify_df

In [ ]:

def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except mysql.connector.OperationalError as e:
        raise e
    else:
        print('Connected!')
    return conn  

In [ ]:
host_name = 'localhost'
dbname = "Spotify_data"
port = '5432'
username = 'postgres'
password = 'llbastian_torresll'
conn= None

conn = connect_to_db(host_name, dbname, port, username, password)

In [ ]:
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS songs(
                        song_names VARCHAR(200) PRIMARY KEY,
                        artist_names VARCHAR(200) NOT NULL,
                        popularity INT NOT NULL,
                        acousticness FLOAT NOT NULL,
                        danceability FLOAT NOT NULL,
                        duration_ms INT NOT NULL,
                        energy FLOAT NOT NULL,
                        liveness FLOAT NOT NULL,
                        loudness FLOAT NOT NULL,
                        mode INT NOT NULL,
                        speechiness FLOAT NOT NULL,
                        instrumentalness FLOAT NOT NULL,
                        tempo FLOAT NOT NULL, 
                        valence FLOAT NOT NULL
                        );""")

    curr.execute(create_table_command)

In [ ]:
#write insert command 
def insert_into_table(curr,song_names, artist_names, popularity, acousticness, danceability, duration_ms, energy, liveness, loudness, mode, speechiness, instrumentalness, tempo, valence):
    insert_into_songs = ("""INSERT INTO songs (song_names, artist_names, popularity, acousticness, danceability, duration_ms, energy, liveness, loudness, mode, speechiness, instrumentalness, tempo, valence)
    VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""")

    row_to_insert = (song_names, artist_names, popularity, acousticness, danceability, duration_ms, energy, liveness, loudness, mode, speechiness, instrumentalness, tempo, valence)

    curr.execute(insert_into_songs, row_to_insert)

In [ ]:
#check_song_exist
def check_if_song_exists(curr, song_names, artist_names):
    query = ("""SELECT song_names FROM songs WHERE song_names = %s AND artist_names = %s""")
    curr.execute(query, (song_names, artist_names,))
    
    return curr.fetchone() is not None 

In [ ]:
def append_from_df_to_db(curr, spotify_df):
    for i, row in spotify_df.iterrows():
          insert_into_table(curr, row['song_names'], row['artist_names'], row['popularity'], row['acousticness'], row['danceability'], row['duration_ms'], row['energy'], row['liveness'], row['loudness'], row['mode'], row['speechiness'], row['instrumentalness'], row['tempo'], row['valence'])

In [ ]:
#Check if song exist in the db
def update_db(curr, spotify_df):
    tmp_df = pd.DataFrame(columns=['song_names', 'artist_names', 'popularity', 'acousticness', 'danceability', 'duration_ms', 'energy', 'liveness', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'tempo', 'valence'])

    for i, row in spotify_df.iterrows():
          if check_if_song_exists(curr, row['song_names'], row['artist_names']): #If song already exists then we'll pass
            pass
          else:  #The song doesn't exists so we'll append to the db table
            tmp_df = tmp_df.append(row)

    return tmp_df

In [ ]:
curr = conn.cursor()

In [ ]:
create_table(curr)

In [ ]:
new_songs_df = update_db(curr, spotify_df)
conn.commit()

In [ ]:
new_songs_df

In [ ]:
append_from_df_to_db(curr, new_songs_df)
conn.commit()